In [23]:
#creation database

import sqlite3

# connexion (+ creation de la base si elle n'existe pas)
connexion = sqlite3.connect('Database.db')
cursor = connexion.cursor()
# Creation des tables dans la base de donnée
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS d_entreprise (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        entreprise_nom TEXT,
        entreprise_description TEXT
    );

    CREATE TABLE IF NOT EXISTS h_departement (
        departement TEXT PRIMARY KEY,
        departement_nom,
        region TEXT
    );
    
    CREATE TABLE IF NOT EXISTS d_ville (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ville TEXT,
        latitude REAL,
        longitude REAL,
        code_postal TEXT,
        departement TEXT,
        FOREIGN KEY (departement) REFERENCES h_departement (departement)
    );

    CREATE TABLE IF NOT EXISTS offres (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        intitule TEXT,
        description_offre TEXT,
        date_creation TEXT,
        salaire_min_annuel TEXT,
        salaire_max_annuel TEXT,
        salaire_annuel_mean TEXT,
        qualification_libelle TEXT,
        experience TEXT,
        type_contrat TEXT,
        secteur_activite TEXT,
        id_ville INTEGER,
        id_entreprise INTEGER,
        FOREIGN KEY (id_ville) REFERENCES d_ville (id),
        FOREIGN KEY (id_entreprise) REFERENCES d_entreprise (id)
    );
''')
# Commit changes and close the connection
connexion.commit()
connexion.close()

In [24]:
#connexion.close()

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('Jobs_V3.csv')

In [27]:
df.head(3)

,intitule,description_offre,dateCreation,typeContrat,experienceExige,secteurActiviteLibelle,qualificationLibelle,departement,ville,experienceOuiNon,...,salaire,code_postal,latitude,longitude,department_name,region,nouvelle_colonne,salaire_min_annuel,salaire_max_annuel,salaire_annuel_mean
0,Consultant(e) Senior Data Engineer F/H - ADN E...,Poste et missions\n \n \n \n- \nParticiper à...,30/12/2023,CDI,E,Activités des agences de travail temporaire,NaN,Ile-de-France,NaN,oui,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Senior Data Engineer F/H (H/F),"Le saviez-vous ?\n\nNous rejoindre, c'est rejo...",29/12/2023,CDI,E,Autres activités informatiques,NaN,91,MASSY,oui,...,NaN,76270.0,49.687882,1.395968,seine-maritime,normandie,NaN,NaN,NaN,NaN
2,Data Engineer H/F,"Rejoindre le Groupe IRCEM, c'est participer à ...",29/12/2023,CDI,E,NaN,NaN,59,ROUBAIX,oui,...,NaN,59100.0,50.688369,3.181903,nord,hauts-de-france,NaN,NaN,NaN,NaN


In [28]:
df = df.rename(columns={'dateCreation':'date_creation', 'qualificationLibelle':'qualification_libelle', 'experienceOuiNon':"experience", 'typeContrat':'type_contrat', 'secteurActiviteLibelle':'secteur_activite', 'codePostal':'code_postal', 'department_name':'departement_nom' })

In [29]:
#temporaire, pour utiliser le departement
df['departement'] = df['departement_nom']

In [30]:
df.columns

Index(['intitule', 'description_offre', 'date_creation', 'type_contrat',
       'experienceExige', 'secteur_activite', 'qualification_libelle',
       'departement', 'ville', 'experience', 'entreprise_nom',
       'entreprise_description', 'salaire', 'code_postal', 'latitude',
       'longitude', 'departement_nom', 'region', 'nouvelle_colonne',
       'salaire_min_annuel', 'salaire_max_annuel', 'salaire_annuel_mean'],
      dtype='object')

In [31]:
df['entreprise_nom'].unique().shape

(172,)

In [32]:
df['entreprise_description'].unique().shape

(200,)

In [33]:
df[['entreprise_nom', 'entreprise_description']].drop_duplicates()

,entreprise_nom,entreprise_description
0,ADN Emploi,ADN Emploi Lille est une agence de recrutement...
1,NaN,"Nous rejoindre, c'est rejoindre l'un des leade..."
2,IRCEM,NaN
3,CPF,"Depuis 2002, Amorino émerveille les gourmands ..."
4,NaN,Ministère de l'Intérieur
...,...,...
406,EURO-INFORMATION DEVELOPPEMENTS,"Euro-Information, filiale technologique de Cré..."
407,CNAF,Vous souhaitez rejoindre une équipe expériment...
408,DIRECTION DU NUMÉRIQUE DES MINISTÈRES SOCIAUX,Vous souhaitez intégrer une Direction numériqu...
409,CNAF,Vous souhaitez rejoindre une équipe expériment...


In [34]:
# print(df[['departement_nom']].drop_duplicates()) 
print(df[['departement']].drop_duplicates()) 

               departement
0                      NaN
1           seine-maritime
2                     nord
3             val-de-marne
5                   loiret
6         bouches-du-rhône
7                    rhône
9         loire-atlantique
10                 gironde
12                bas-rhin
13            eure-et-loir
14                calvados
15                 hérault
16                   paris
19          hauts-de-seine
20           haute-garonne
28                  savoie
51                   drôme
53                  nièvre
58            haute-savoie
64         alpes-maritimes
68                yvelines
72                vaucluse
74                    aude
76         ille-et-vilaine
126            deux-sèvres
139                  isère
146                    var
147                 landes
155   pyrénées-atlantiques
174                   gard
186                essonne
192                aveyron
213                 sarthe
222  territoire-de-belfort
228                moselle
2

In [35]:
connexion = sqlite3.connect('Database.db')

# insertion des données dans les tables d_entreprise, h_departement, d_ville
# on insert pas les id pour le moment
# d_entreprise
print(df[['entreprise_nom', 'entreprise_description']].drop_duplicates().head(3))

df[['entreprise_nom', 'entreprise_description']].drop_duplicates().to_sql('d_entreprise', connexion, if_exists='append', index=False)

# h_departement
print(df[['departement']].drop_duplicates()) #on pourra ajouter la région dans cette table plus tard
df[['departement','region']].drop_duplicates().to_sql('h_departement', connexion, if_exists='append', index=False)

# d_ville
print(df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().head(3))
df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().to_sql('d_ville', connexion, if_exists='append', index=False)


  entreprise_nom                             entreprise_description
0     ADN Emploi  ADN Emploi Lille est une agence de recrutement...
1            NaN  Nous rejoindre, c'est rejoindre l'un des leade...
2          IRCEM                                                NaN
               departement
0                      NaN
1           seine-maritime
2                     nord
3             val-de-marne
5                   loiret
6         bouches-du-rhône
7                    rhône
9         loire-atlantique
10                 gironde
12                bas-rhin
13            eure-et-loir
14                calvados
15                 hérault
16                   paris
19          hauts-de-seine
20           haute-garonne
28                  savoie
51                   drôme
53                  nièvre
58            haute-savoie
64         alpes-maritimes
68                yvelines
72                vaucluse
74                    aude
76         ille-et-vilaine
126            deux-sèvres

132

In [36]:
#df[['departement','departement_nom','region']].drop_duplicates()
df[['departement','region']].drop_duplicates()

,departement,region
0,NaN,NaN
1,seine-maritime,normandie
2,nord,hauts-de-france
3,val-de-marne,île-de-france
5,loiret,centre-val de loire
6,bouches-du-rhône,provence-alpes-côte d'azur
7,rhône,auvergne-rhône-alpes
9,loire-atlantique,pays de la loire
10,gironde,nouvelle-aquitaine
12,bas-rhin,grand est


In [37]:
df.loc[:,'ville']

0                 NaN
1               MASSY
2             ROUBAIX
3                ORLY
4      HAUTS DE SEINE
            ...      
406        STRASBOURG
407          PARIS 14
408          PARIS 15
409          PARIS 14
410           BELFORT
Name: ville, Length: 411, dtype: object

In [38]:
# d_ville
print(df[['ville', 'latitude', 'longitude', 'code_postal', 'departement']].drop_duplicates().head(3))

     ville   latitude  longitude  code_postal     departement
0      NaN        NaN        NaN          NaN             NaN
1    MASSY  49.687882   1.395968      76270.0  seine-maritime
2  ROUBAIX  50.688369   3.181903      59100.0            nord


In [39]:
''' la fonction map est comme la fonction apply de R, 
il applique une fonction sur chaque ligne de la dataframe.
Nous nous en servons pour réccupérer les id de chaque entreprise dans d_entreprise,
ces id seront placés dans la colonne id_entreprise de la table offres
'''

# creation de id_entreprise
company_mapping = pd.read_sql('SELECT id,entreprise_nom,entreprise_description FROM d_entreprise', connexion)
df = pd.merge(df, company_mapping, how='left', on=['entreprise_nom', 'entreprise_description'], suffixes=('', '_ent'))
df.rename(columns={'id': 'id_entreprise'}, inplace=True)

ville_mapping = pd.read_sql('SELECT id,ville FROM d_ville', connexion)
df['id_ville'] = df['ville'].map(ville_mapping.set_index(['ville'])['id'])


In [40]:
df['id_entreprise'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [41]:
# Insertion des données dans la table offre
df[['intitule', 'description_offre', 'date_creation', 'salaire_min_annuel', 'salaire_max_annuel', 'salaire_annuel_mean', 'qualification_libelle',
   'experience', 'type_contrat', 'secteur_activite', 'id_ville', 'id_entreprise']].to_sql('offres', connexion, if_exists='append', index=False)

# Commit et fermeture de la connexion
connexion.commit()
connexion.close()


In [44]:
# connexion.close()

In [46]:
# accès à la table departement pour afficher son contenu
connexion = sqlite3.connect('Database.db')
cursor = connexion.cursor()

# selectionner toutes les colonnes de la table d_ville
query = 'SELECT * FROM offres'
cursor.execute(query)

# placer la selection dans une variable
rows = cursor.fetchall()

# parcourir la variable pour afficher les données
for row in rows:
    print(row)

connexion.close()

(1, 'Consultant(e) Senior Data Engineer F/H - ADN Emploi (H/F)', "Poste et missions\n \n \n  \n-  \nParticiper à la conception de solutions : choix d'outils, proposition de solutions alternatives et concevoir ou participer à la construction de solutions de data engineering\n  \n-  \nAssister le client au cadrage du projet, et participer au design fonctionnel et technique\n  \n-  \nStructurer et planifier la réalisation des développements et proposer des améliorations continues et d'optimisation techniques\n  \n-  \nCollaborer à l'intégration des produits dans le SI avec l'architecte technique.\n  \n-  \nParticipation à la réalisation de développements complexes\n  \n-  \nAssurer le suivi et la qualité des développements ainsi que la réalisation du projet et d'accompagner les développeurs à écrire du code propre, qualitatif et conforme aux standards de l'équipe\n  \n-  \nMaîtriser les techniques et les méthodes Agiles\n  \n-  \nExperience dans un environnement Big Data (Spark, Hive, Imp